# Overview

Let's implement backpropgation manually.

In [1]:
import random
import torch

# read in all the words
with open('/kaggle/input/character-lm-without-framework/names.txt', 'r', encoding='utf-8') as f:
    words=f.read()

words=words.splitlines()

# build the vocabulary of characters and 
chars=sorted(list(set(''.join(words))))

stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos={i:s for s,i in stoi.items()}
vocab_size=len(itos)
block_size=3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
    X,Y=[],[]
    
    for w in words:
        context=[0]*block_size
        for ch in w+'.':
            ix=stoi[ch]
            X.append(context)
            Y.append(ix)
            context=context[1:]+[ix] # crop and append
            
    X=torch.tensor(X)
    Y=torch.tensor(Y)
    print(X.shape, Y.shape)
    return X,Y

random.seed(42)
random.shuffle(words)
n1=int(0.8*len(words))
n2=int(0.9*len(words))

Xtr, Ytr=build_dataset(words[:n1])     # 80%
Xdev, Ydev=build_dataset(words[n1:n2]) # 10%
Xte, Yte=build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [2]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt,t):
    ex=torch.all(dt==t.grad).item() # make sure all the elements are exactly equal and then converting a single boolean value
    # make sure they aren't exactly equal, they are approximately equal(there may some floating point issues: floating point arithmetic)
    app=torch.allclose(dt, t.grad)
    # check what is the highest difference 
    maxdiff=(dt-t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [3]:
# initialization
n_embd=10 # the dimensionality of the character embedding vectors
n_hidden=64 # the number of neurons in the hidden layer of the MLP

g=torch.Generator().manual_seed(2147483647) # for reproducibility
C=torch.randn((vocab_size, n_embd), generator=g) # embedding table for the characters
# Layer 1
W1=torch.randn((n_embd*block_size, n_hidden), generator=g)*(5/3)/((n_embd* block_size)**0.5)
b1=torch.randn(n_hidden,            generator=g) # b1 is useless here, it aims for us to check if the implement is correct
# Layer 2
W2=torch.randn((n_hidden, vocab_size), generator=g)*0.1
b2=torch.randn(vocab_size, generator=g)*0.1
# batchnorm parameters
bngain=torch.randn((1, n_hidden))*0.1+1.0
bnbias=torch.randn((1, n_hidden))*0.1

# Note: we initializating many of thrse parameters in non-standard ways
# becuase sometimes initializating with e.g all zeros coudl mask an incorrect implementation of the backward pass

parameters=[C,W1,b1,W2,b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
    p.requires_grad=True

4137


In [4]:
batch_size=32
n=batch_size
# construct a minibatch
ix=torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb=Xtr[ix], Ytr[ix] # batch X,Y

In [5]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb=C[Xb] # embed the characters into vectors
print(emb.shape)
embcat=emb.view(emb.shape[0],-1) # concatenate the vectors
print(embcat.shape)
print(W1.shape)
# Linear layer 1
hprebn=embcat@W1+b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani=1/n*hprebn.sum(0, keepdim=True)
bndiff=hprebn-bnmeani
bndiff2=bndiff**2
bnvar=1/(n-1)*(bndiff2).sum(0, keepdim=True)
bnvar_inv=(bnvar+1e-5)**-0.5
bnraw=bndiff*bnvar_inv
hpreact=bngain*bnraw+bnbias
# non-linearity
h=torch.tanh(hpreact) # hidden layer
# linear layer 2
logits=h@W2+b2 # output layer


# cross entropy loss (same as F.cross_entropy(logits, Yb))

logit_maxes=logits.max(1, keepdim=True).values
norm_logits=logits-logit_maxes # subtract max for nemerical stability
counts=norm_logits.exp() # keep the logits on small values avoid numerical issues
counts_sum=counts.sum(1, keepdims=True)


# normalization all the counts to create our probabilities
counts_sum_inv=counts_sum**-1 # if we use (1.0/counts_sum) instead then we can't get backprop to be bit exact...
probs=counts*counts_sum_inv
logprobs=probs.log()
loss=-logprobs[range(n), Yb].mean()

# pytorch backward pass
for p in parameters:
    p.grad=None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, norm_logits, logit_maxes, logits, h, hpreact, bnraw, bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani, embcat, emb]:
    t.retain_grad()
loss.backward()
loss

torch.Size([32, 3, 10])
torch.Size([32, 30])
torch.Size([30, 64])


tensor(3.3405, grad_fn=<NegBackward0>)

# Implement backprop manually

`dlogprobs` will hold the derivative of the loss with respect to all the elements of log props. It should also be an array that size same to `logprobs` because we want the derivative loss with respect to all of its elements.

In [6]:
logprobs.shape

torch.Size([32, 27])

## How does log props influence the loss?

We see the loss equal to `loss=-logprobs[range(n), Yb].mean()`. Yb here is an array includes all the correct indices. So, here each single row of logprobs, we are plucking out the index of column specified by the tensor `Yb`.

In [7]:
Yb[0]

tensor(8)

In [8]:
logprobs[0, Yb[0]]==logprobs[0][Yb[0]]

tensor(True)

**Here we plugs out all those log probabilities of the correct next character in a sequence**

In [9]:
logprobs[range(n), Yb]

tensor([-4.0807, -2.9965, -3.6843, -3.3662, -4.1294, -3.4584, -3.2458, -3.9973,
        -3.1246, -4.2837, -3.1347, -1.6126, -2.7956, -2.9740, -2.8601, -3.1692,
        -3.9216, -2.9884, -3.6308, -3.3777, -2.8634, -3.0112, -4.2735, -4.0063,
        -3.6032, -2.8557, -3.0109, -3.8330, -2.7471, -3.4568, -3.2837, -3.1193],
       grad_fn=<IndexBackward0>)

## Deriviative represent in number

`loss=-(a+b+c)/3=-1/3a+ -1/3b+-1/3c` and `dloss/da=-1/3` n is 3 here.

Here, we have n=32(batch_size). So, `dloss/da=-1/n`.


### What about the other elements inside logprobs?

Here n is equal to `batch_size`, it is less than the length of logprobs(32,27). Only 32 of logprobs participate in the loss calculation.

### What's the derivative of all the other most of elements that don't get plucked out here?

Their gradient intuitively is zero because they didn't participate in the loss. So, the most of these numbers inside this tensor doesn't feed into the loss. And if you were to change these numbers then the loss doen't change which is the equivalent of way of the derovative of the loss with respect to them is zero. They don't impact it.



In [10]:
# size of dlogprobs is always going to be equal to logprobs
# don't hardcode the number
dlogprobs=torch.zeros_like(logprobs)

# set the dloss/da=-1/n inside exactly these locations.
dlogprobs[range(n), Yb]=-1.0/n
dlogprobs[0]

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
        -0.0312,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000])

## Check if the value is correct

In [11]:
# our result equal to what pytorch calculated to be lockprops.grad in its back propagation
cmp('logprobs', dlogprobs, logprobs)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0


## Props of a lognode

Based on `logprobs=probs.log()`, the props of the log node will be the **local derivative** of that individual operation log times the output(here is `dlogprobs`).

Based on $\frac{d}{dx}(log(x))=\frac{1}{x}$, x here is probs.

### Chain rule

In calculus, it is a formula that express the derivative of the composition of two differentiable functions f and g in terms of the derivatives of f and g.

if $h=f*g$ is the function such that $h(x)=f(g(x))$ for every x, then the chain rule is

$$h'(x)=f'(g(x))g'(x)$$ or, equivalently

$$h'=(f*g)'=(f'*g)*g'$$

### Lagrange's notation

If `f` is a function, then its derivative evaluated at `x` is written $f'(x)$.

In [12]:
#dlogprobs is chain rule
dprobs=(1.0 /probs)*dlogprobs # if the probs is incorrect, we will boost the gradient thorugh this

In [13]:
cmp('probs', dprobs, probs)

probs           | exact: True  | approximate: True  | maxdiff: 0.0


## Derivative the counts_sum_inv

Be cautions of the shape of these tensors.

```
c=a*b, but with tensors:
* a[3x3] b[3,1]
* a11*b1 a12*b1 a13*b1
* a21*b2 a22*b2 a23*b2
* a31*b3 a32*b3 a33*b3
* c[3x3]
```

In [14]:
counts.shape, counts_sum_inv.shape

(torch.Size([32, 27]), torch.Size([32, 1]))

In [15]:
dcounts_sum_inv=(counts*dprobs).sum(1, keepdim=True)

In [16]:
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0


## Deriviative dcounts and dcounts_sum

$$\frac{d}{dx}(\frac{1}{x})=-\frac{1}{x^2}$$

In [17]:
dcounts=counts_sum_inv*dprobs
dcounts_sum=(-counts_sum**-2)* dcounts_sum_inv

In [18]:
cmp('counts_sum', dcounts_sum, counts_sum)

counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0


## Derivative counts

In [19]:
dcounts+=torch.ones_like(counts)*dcounts_sum

In [20]:
cmp('counts', dcounts, counts)

counts          | exact: True  | approximate: True  | maxdiff: 0.0


## Derivative norm_logits

In [21]:
# dnorm_logits=(norm_logits.exp())
dnorm_logits=counts*dcounts

In [22]:
cmp('norm_logits', dnorm_logits, norm_logits)

norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0


## Derivative logits and logits_maxes



In [23]:
dilogits=dnorm_logits.clone()
dlogit_maxes=(-dnorm_logits).sum(1, keepdim=True)

# Acknowledgement

* https://en.wikipedia.org/wiki/Chain_rule
* https://en.wikipedia.org/wiki/Notation_for_differentiation#Lagrange's_notation